# UTILS

In [1]:
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import numpy as np

def make_pairs(images, labels):
    # initialize two empty lists to hold the (image, image) pairs and
    # labels to indicate if a pair is positive or negative
    pairImages = []
    pairLabels = []
    
    # calculate the total number of classes present in the dataset
    # and then build a list of indexes for each class label that
    # provides the indexes for all examples with a given label
    numClasses = len(np.unique(labels))
    idx = [np.where(labels == i)[0] for i in range(0, numClasses)]
    
    # loop over all images
    for idxA in range(len(images)):
        # grab the current image and label belonging to the current
        # iteration
        currentImage = images[idxA]
        label = labels[idxA]
        
        # randomly pick an image that belongs to the *same* class
        # label
        idxB = np.random.choice(idx[label])
        posImage = images[idxB]
        
        # prepare a positive pair and update the images and labels
        # lists, respectively
        pairImages.append([currentImage, posImage])
        pairLabels.append([1])
        
        # grab the indices for each of the class labels *not* equal to
        # the current label and randomly pick an image corresponding
        # to a label *not* equal to the current label
        negIdx = np.where(labels != label)[0]
        negImage = images[np.random.choice(negIdx)]
        
        # prepare a negative pair of images and update our lists
        pairImages.append([currentImage, negImage])
        pairLabels.append([0])
        
    # return a 2-tuple of our image pairs and labels
    return (np.array(pairImages), np.array(pairLabels))

def euclidean_distance(vectors):
    # unpack the vectors into separate lists
    (featsA, featsB) = vectors
    
    # compute the sum of squared distances between the vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1,
        keepdims=True)
    
    # return the euclidean distance between the vectors
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def plot_training(H, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(H.history["loss"], label="train_loss")
    plt.plot(H.history["val_loss"], label="val_loss")
    # plt.plot(H.history["accuracy"], label="train_acc")
    # plt.plot(H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

# CONFIG

In [2]:
# import the necessary packages
import os

# specify the shape of the inputs for our network
IMG_SHAPE = (28, 28, 1)

# specify the batch size and number of epochs
BATCH_SIZE = 128
EPOCHS = 20

# define the path to the base output directory
BASE_OUTPUT = "Contrastive_Loss_output"
# use the base output path to derive the path to the serialized
# model along with training history plot
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "siamese_model"])
PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plot.png"])

# TEST

In [3]:
# import the necessary packages
from tensorflow.keras.models import load_model
from imutils.paths import list_images
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [4]:
from tensorflow.keras.datasets import mnist
print("[INFO] loading test dataset...")
(trainX, trainY), (testX, testY) = mnist.load_data()
testX = testX / 255.0

[INFO] loading test dataset...


In [5]:
(pairTest, labelTest) = make_pairs(testX, testY)
print(pairTest.shape)
x = pairTest[:10,:]
# x = np.reshape(x, (x.shape, 2, 28, 28, 1))
print(x.shape)

(20000, 2, 28, 28)
(10, 2, 28, 28)


In [6]:
# load the model from disk
print("[INFO] loading siamese model...")
model = load_model(MODEL_PATH)

[INFO] loading siamese model...


SystemError: unknown opcode

In [ ]:
rd = np.arange(20000)
for i in range (0,2):
  a = np.random.choice(rd)
  origA = pairTest[a,0,:]
  origB = pairTest[a,1,:]
  x1 = np.reshape(pairTest[a,0,:],(1,28,28))
  x2 = np.reshape(pairTest[a,1,:],(1,28,28))
  preds = model.predict([x1, x2])

  proba = preds[0][0] 
  # initialize the figure
  fig = plt.figure("Pair #{}".format(i + 1), figsize=(4, 2))
  plt.suptitle("Distance: {:.2f}".format(proba))
  # show first image
  ax = fig.add_subplot(1, 2, 1)
  plt.imshow(origA, cmap=plt.cm.gray)
  plt.axis("off")
  # show the second image
  ax = fig.add_subplot(1, 2, 2)
  plt.imshow(origB, cmap=plt.cm.gray)
  plt.axis("off")
  # show the plot
  plt.show()